In [27]:
import pandas as pd
import matplotlib.pyplot as plt
import glob
import re
import time
import warnings
import numpy as np
from nltk.corpus import stopwords
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import normalize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.manifold import TSNE
import seaborn as sns
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics.classification import accuracy_score, log_loss
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from imblearn.over_sampling import SMOTE
from collections import Counter
from scipy.sparse import hstack
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold 
from collections import Counter, defaultdict
from sklearn.calibration import CalibratedClassifierCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
import math

from sklearn import model_selection
from sklearn.linear_model import LogisticRegression


from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from difflib import SequenceMatcher

C:\Users\lduggirala\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:143: FutureWarning: The sklearn.metrics.classification module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)


In [28]:
import nltk
from nltk.corpus import stopwords
from textblob import TextBlob
from nltk.tokenize import word_tokenize,sent_tokenize
import spacy
from spacy import displacy
from nltk.stem.snowball import PorterStemmer, SnowballStemmer
from nltk import WordNetLemmatizer
from string import punctuation
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

stop = stopwords.words('english')
nlp = spacy.load("en_core_web_sm")
snow = SnowballStemmer('english')
lemma = WordNetLemmatizer()

cv = CountVectorizer()
tfidf = TfidfVectorizer()

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\lduggirala\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\lduggirala\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [29]:
txt = pd.read_excel('AI_EVENTS_ALL_AM.xlsx',sheet_name='TOTAL_DATA_ALL_AI')

In [30]:
#Text Preprocessing
def text_preprocessing(text):
    txt = str(text).lower()
    txt = re.sub(r'[^\w\s]','',txt)
    txt = ' '.join([x for x in txt.split() if x not in stop])
    #txt = str(TextBlob(txt).correct())
    #txt = ' '.join([snow.stem(txt)])
    return txt

In [31]:
txt1 = txt[txt['class'].isin(['NE','PC'])]
txt2 = txt[txt['class'].isin(['AE'])][0:15000]
txt = pd.concat([txt1,txt2])

In [32]:
txt['ppd_text'] = txt['text'].map(text_preprocessing)

In [33]:
# Importing spacy
import spacy

# Loading model
nlp = spacy.load('en_core_web_sm',disable=['parser', 'ner'])

# Lemmatization with stopwords removal
txt['lemmatized']=txt['ppd_text'].apply(lambda x: ' '.join([token.lemma_ for token in list(nlp(x)) if (token.is_stop==False)]))

In [34]:
txt['numer_class'] = txt['class'].apply(lambda x:0 if x=='AE' else(1 if x=='PC' else 2))

X = txt['lemmatized']
Y = txt['numer_class']

Y=Y.astype(int)

In [35]:
#Train-CV-Test Split
# split the data into test and train by maintaining same distribution of output varaible 'y_true' [stratify=y_true]
X_train, test_df, y_train, y_test = train_test_split(X, Y, test_size=0.2)
# split the train data into train and cross validation by maintaining same distribution of output varaible 'y_train' [stratify=y_train]
train_df, cv_df, y_train, y_cv = train_test_split(X_train, y_train, stratify=y_train, test_size=0.2)

In [36]:
import gensim
from gensim.models import Word2Vec
from gensim.models import Phrases
from nltk.tokenize import sent_tokenize, word_tokenize
import warnings
import multiprocessing
from time import time

In [52]:
data = [word_tokenize(word) for word in txt['lemmatized'] if len(word)>1]

# Creating the model and setting values for the various parameters
num_features = 800  # Word vector dimensionality
min_word_count = 3 # Minimum word count
num_workers = 4     # Number of parallel threads
context = 4       # Context window size
downsampling = 1e-3 # (0.001) Downsample setting for frequent words

# Initializing the train model
from gensim.models import word2vec
print("Training model....")
model = word2vec.Word2Vec(data,\
                          workers=num_workers,\
                          vector_size=num_features,\
                          min_count=min_word_count,\
                          window=context,
                          sample=downsampling)

# To make the model memory efficient
model.init_sims(replace=True)

# Saving the model for later use. Can be loaded using Word2Vec.load()
model_name = "300features_40minwords_10context"
model.save(model_name)

Training model....


<ipython-input-52-7c30f70dade2>:21: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


In [53]:
model.wv.similarity('patient','injection')

0.4100232

In [54]:
model.wv.most_similar("patient")

[('mother', 0.770642101764679),
 ('pt', 0.7690114974975586),
 ('nurse', 0.7496442794799805),
 ('caller', 0.7324281930923462),
 ('husband', 0.7311583757400513),
 ('friend', 0.7297000885009766),
 ('wife', 0.7169663906097412),
 ('daughter', 0.7074563503265381),
 ('today', 0.6964541077613831),
 ('father', 0.681010365486145)]

In [55]:
# Keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout, Embedding, Flatten, Conv1D, MaxPooling1D, LSTM
from keras import utils
from keras.callbacks import ReduceLROnPlateau, EarlyStopping

# nltk
import nltk
from nltk.corpus import stopwords
from  nltk.stem import SnowballStemmer

In [56]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(pd.DataFrame(train_df)['lemmatized'])

vocab_size = len(tokenizer.word_index) + 1
print("Total words", vocab_size)

Total words 20633


In [57]:
x_train = pad_sequences(tokenizer.texts_to_sequences(pd.DataFrame(train_df)['lemmatized']), maxlen=num_features)
x_cv = pad_sequences(tokenizer.texts_to_sequences(pd.DataFrame(cv_df)['lemmatized']), maxlen=num_features)
x_test = pad_sequences(tokenizer.texts_to_sequences(pd.DataFrame(test_df)['lemmatized']), maxlen=num_features)

In [93]:
#y_train1 = np.array(y_train).reshape(-1,1)
#y_cv1 = np.array(y_cv).reshape(-1,1)
#y_test1 = np.array(y_test).reshape(-1,1)

from tensorflow import keras
y_train1 = keras.utils.to_categorical(np.array(y_train),3)
y_cv1 = keras.utils.to_categorical(np.array(y_cv),3)
y_test1 = keras.utils.to_categorical(np.array(y_test),3)

In [76]:
print("x_train", x_train.shape)
print("y_train", y_train1.shape)
print()
print("x_cv", x_cv.shape)
print("y_cv", y_cv1.shape)
print()
print("x_test", x_test.shape)
print("y_test", y_test1.shape)

x_train (27332, 800)
y_train (27332, 1)

x_cv (6834, 800)
y_cv (6834, 1)

x_test (8542, 800)
y_test (8542, 1)


In [77]:
embedding_matrix = np.zeros((vocab_size, num_features))
for word, i in tokenizer.word_index.items():
    if word in model.wv:
        embedding_matrix[i] = model.wv[word]
print(embedding_matrix.shape)

(20633, 800)


In [78]:
embedding_layer = Embedding(vocab_size, num_features, weights=[embedding_matrix], input_length=num_features, trainable=False)

In [97]:
model = Sequential()
model.add(embedding_layer)
model.add(Dropout(0.5))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(3, activation='softmax'))

model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 800, 800)          16506400  
_________________________________________________________________
dropout_3 (Dropout)          (None, 800, 800)          0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 100)               360400    
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 303       
Total params: 16,867,103
Trainable params: 360,703
Non-trainable params: 16,506,400
_________________________________________________________________


In [98]:
model.compile(loss='categorical_crossentropy',
              optimizer="adam",
              metrics=['accuracy'])

In [100]:
# KERAS
SEQUENCE_LENGTH = 800
EPOCHS = 32
BATCH_SIZE = 128

In [101]:
history = model.fit(x_train, y_train1,
                    batch_size=BATCH_SIZE,
                    epochs=EPOCHS,
                    validation_split=0.1,
                    verbose=1,
                    #callbacks=callbacks
                    )

Epoch 1/32
 35/193 [====>.........................] - ETA: 2:59:14 - loss: 0.7166 - accuracy: 0.7080

KeyboardInterrupt: 